This script was created to help create the Location Dimension for our crime fact Table. In this script we:
1. Create a dataframe of the location features from Vancouver Crime Dataset
2. Create a dataframe from the Vancouver Census Data retrieved from Vancouver Open Data
3. Merge these 2 dataframes
4. Repeat step 1 for Denver
5. Repeat step 2 for Denver Open Data
6. Repeat step 3 for Denver
7. Merge Vancouver and Denver Dataframes
8. Export as CSV to be imported to SQL database

In [1]:
import pandas as pd
import numpy as np
#Vancouver Data
#Vancouver
van = pd.read_csv("crimeVan.csv", encoding="UTF-8")
row_count = len(van)

cityID_col = np.full(row_count,'1')
cityName_col = np.full(row_count,'VANCOUVER')
address_col = van["HUNDRED_BLOCK"]
neighbourhood_col = van["NEIGHBOURHOOD"]
latitude_col = van["Latitude"]
longitude_col = van["Longitude"]
x_col = van["X"]
y_col = van["Y"]
#You fill
pop_col = np.full(row_count,'0')
averageHH_col = np.full(row_count,'0')
unemployment_col = np.full(row_count,'NOT GIVEN')
#null
poverty_col = np.full(row_count,'NOT GIVEN')

dfvan = pd.DataFrame({'cityID':cityID_col, 'cityName':cityName_col, 
                          'address':address_col, 'neighbourhood':neighbourhood_col, 'latitude':latitude_col,
                         'longitude':longitude_col,'x':x_col,'y':y_col,'neighbourhood':neighbourhood_col,
                      'povertyRate':poverty_col})

dfvan = dfvan.drop_duplicates()
dfvan = dfvan.fillna('NOT GIVEN')
dfvan["address"] = dfvan["address"].replace(to_replace='OFFSET TO PROTECT PRIVACY',
                                                  value="NOT GIVEN")

#Vancouver Census Data
cols=[0,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25]
vanCensus = pd.read_csv("CensusLocalAreaProfiles2016.csv",encoding="latin1")

rows=[' Total - Age groups and average age of the population - 100% data ','Employment rate',
      'Unemployment rate','    Average total income in 2015 among recipients ($)','  Average monthly shelter costs for owned dwellings ($)']
vanCen=vanCensus[vanCensus['Variable'].isin(rows)]

nh = list(vanCen.columns.values)
nh = nh[2:]
header = vanCen['Variable'].tolist()

values = vanCen.values.tolist()
rows=[]
for row in values:
    rows.append(row[2:])
    
for i in range(len(nh)):
    nh[i]=nh[i][:-1]
nh[0]= "Arbutus Ridge"
nh[1]= "Central Business District"
nh[19]= "Victoria-Fraserview"

nh = nh[:-2]
rows[0]=rows[0][:-2]
rows[1]=rows[1][:-2]
rows[3]=rows[3][:-2]

nh.append('Musqueam')
rows[0].append('0')
rows[1].append('0')
rows[3].append('0')

nh.append('Stanley Park')
rows[0].append('0')
rows[1].append('0')
rows[3].append('0')

nh.append('NOT GIVEN')
rows[0].append('0')
rows[1].append('0')
rows[3].append('0')
    
df = pd.DataFrame({'neighbourhood':nh,'population':rows[0],'averageHHIncome':rows[1],
                   'unemploymentRate':rows[3]})
for i in range(len(df["population"])):
    if (type(df["population"][i]) is str):
        df["population"][i] = int(df["population"][i].replace(',',''))

#Merge Vancouver with Census
dfvan = pd.merge(dfvan, df, on='neighbourhood')

#Denver Data
d_nhbd = pd.read_csv('denver_nbrhd_2013_2017.csv')
den = pd.read_csv("crimeDen.csv", encoding="UTF-8")

for i in range(len(d_nhbd)):
    d_nhbd['NBHD_NAME'][i] = d_nhbd['NBHD_NAME'][i].replace(' ', '-')
    d_nhbd['NBHD_NAME'][i] = d_nhbd['NBHD_NAME'][i].lower()
    
for i in range(len(d_nhbd)):
    d_nhbd['NBHD_NAME'][i] = d_nhbd['NBHD_NAME'][i].replace('---', '-')
    
nbhd_final = d_nhbd[['NBHD_NAME', 'TTL_POPULATION_ALL', 'AVG_HH_INCOME', 'PCT_POVERTY']].sort_index()   

row_count = len(den)

cityID_col = np.full(row_count,'2')
cityName_col = np.full(row_count,'DENVER')
address_col = den["INCIDENT_ADDRESS"]
latitude_col = den["GEO_LAT"]
longitude_col = den["GEO_LON"]
neighbourhood_col = den["NEIGHBORHOOD_ID"]
x_col = den["GEO_X"]
y_col = den["GEO_Y"]
pop_col = np.full(row_count,'0')
averageHH_col = np.full(row_count,'0')
poverty_col = np.full(row_count,'0')

unemployment_col = np.full(row_count,'0')
nbhd_final.columns = ['neighbourhood', 'population', 'averageHHIncome', 'povertyRate']
dfden = pd.DataFrame({'cityID':cityID_col, 'cityName':cityName_col, 
                          'address':address_col, 'latitude':latitude_col,
                         'longitude':longitude_col,'x':x_col,'y':y_col,'neighbourhood':neighbourhood_col,
                      'unemploymentRate':unemployment_col})

dfden = dfden.drop_duplicates()
dfden["address"] = dfden["address"].fillna('NOT GIVEN')
dfden["neighbourhood"] = dfden["neighbourhood"].fillna('NOT GIVEN')
dfden["latitude"] = dfden["latitude"].fillna('0')
dfden["longitude"] = dfden["longitude"].fillna('0')
for i in range(len(nbhd_final["population"])):
    if (type(nbhd_final["population"][i]) is str):
        nbhd_final["population"][i] = int(nbhd_final["population"][i].replace(',',''))
dfden = pd.merge(dfden, nbhd_final, on='neighbourhood')

#Merge Vancouver and Denver Dataframes
dfVanDen = dfvan.append(dfden)
dfVanDen = dfVanDen.reset_index(drop=True)
dfVanDen.index.names =['locationID']

#Clean data w.r.t SQL constrstraints
dfVanDen["averageHHIncome"] = dfVanDen["averageHHIncome"].replace(to_replace='None',
                                                  value=0)
dfVanDen["povertyRate"] = dfVanDen["povertyRate"].replace(to_replace='NOT GIVEN',
                                                  value=0)
dfVanDen["x"] = dfVanDen["x"].fillna(0)
dfVanDen["y"] = dfVanDen["y"].fillna(0)

dfVanDen.to_csv('LocationDimension.csv', header=True, encoding='utf-8')
print(dfVanDen)

C:\Users\Maxwell\Anaconda3\lib\site-packages\ipykernel_launcher.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Maxwell\Anaconda3\lib\site-packages\ipykernel_launcher.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Maxwell\Anaconda3\lib\site-packages\ipykernel_launcher.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Maxwell\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A f

                                 address averageHHIncome cityID   cityName  \
locationID                                                                   
0                       9XX TERMINAL AVE           31534      1  VANCOUVER   
1                         11XX KEEFER ST           31534      1  VANCOUVER   
2                         11XX KEEFER ST           31534      1  VANCOUVER   
3                         11XX KEEFER ST           31534      1  VANCOUVER   
4                        11XX WILLIAM ST           31534      1  VANCOUVER   
5                          11XX GRANT ST           31534      1  VANCOUVER   
6                       11XX E PENDER ST           31534      1  VANCOUVER   
7                       11XX E PENDER ST           31534      1  VANCOUVER   
8                       11XX E PENDER ST           31534      1  VANCOUVER   
9                     11XX E HASTINGS ST           31534      1  VANCOUVER   
10                    11XX E HASTINGS ST           31534      1 